In [19]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
import numpy as np
import time
import argparse
stemmer = LancasterStemmer()

In [20]:
training_data = []
# createDataSet - create a dataset for training
#def createDataSet():
training_data.append({"class":"greeting", "sentence":"how are you"})
training_data.append({"class":"greeting", "sentence":"how is your day?"})
training_data.append({"class":"greeting", "sentence":"good day"})
training_data.append({"class":"greeting", "sentence":"how is it going today?"})

training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"see you later"})
training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"talk to you soon"})

training_data.append({"class":"sandwich", "sentence":"make me a sandwich"})
training_data.append({"class":"sandwich", "sentence":"can you make a sandwich?"})
training_data.append({"class":"sandwich", "sentence":"having a sandwich today?"})
training_data.append({"class":"sandwich", "sentence":"what's for lunch?"})
print ("%s sentences in training data" % len(training_data))

12 sentences in training data


In [21]:
words = []
classes = []
documents = []
ignore_words = ['?']

for pattern in training_data:
    #tokenize each word in the sentence and append to word list
    w = nltk.word_tokenize(pattern['sentence'])
    words.extend(w)
    documents.append((w, pattern['class']))
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

# remove the duplicate and toLowerCase
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "words", words)


12 documents
3 classes ['goodbye', 'sandwich', 'greeting']
26 words ['yo', 'nic', 'going', 'can', 'good', 'day', 'a', 'you', 'lat', 'today', 'to', 'lunch', 'how', 'soon', 'is', 'me', 'talk', 'for', 'it', 'sandwich', 'see', 'hav', 'what', 'ar', 'mak', "'s"]


In [22]:
# create our training data
training = []
output = []
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # list of tokenized words for the pattern
    bag = []
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    training.append(bag)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)
# sample training/output
i = 0
w = documents[i][0]
print([stemmer.stem(word.lower()) for word in w])
print(training[i])
print(output[i])

['how', 'ar', 'you']
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1]


In [23]:
def sigmoid(x):
    return (1/(1+np.exp(-x)))

In [24]:
# derive_sigmoid - convert output of sigmoid to its derivative
# x - parameter
# math formula x*(1-x)
def derive_sigmoid(x):
    return (x*(1-x))

In [25]:
# clean - tokenize pattern and stem each word
# sentence
def clean(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [26]:
def vectorize(sentence, words):
    # tokenize the pattern
    sentence_words = clean(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return (np.array(bag))

In [27]:
def think(sentence):
    x = vectorize(sentence.lower(), words)
    # input layer (l0) is words
    l0 = x
    # hidden layer (l1) is matrix multiplication
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [41]:
def train(X, y, hidden_neurons= 10, alpha=1, epochs = 50000, dropout = False, dropout_rate = 0.5):
    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_rate if dropout else ''))
    print ("Input matrix: %sx%s  Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)))
    np.random.seed(1)
    
    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons))-1
    synapse_1 = 2*np.random.random((hidden_neurons,len(classes)))-1

    prev_synapse_0_weight = np.zeros_like(synapse_0)
    prev_synapse_1_weight = np.zeros_like(synapse_1)
    
    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)

    for j in iter(range(epochs+1)):
        # think through layer 0, 1, 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))

        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_rate)[0] * (1.0/(1-dropout_rate))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # layer_2 error miss the target value
        layer_2_error = y - layer_2

        if(j % 10000) == 0 and j > 5000:
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print("delta after " + str(j) + " iterations: " + str(np.mean(np.abs(layer_2_error))))
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print("break: ", np.mean(np.abs(layer_2_error)), " > ", last_mean_error)
                break

        # check the direction of target layer 2
        layer_2_delta = layer_2_error * derive_sigmoid(layer_2)
        # layer 1 error contributes to layer 2 error
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # check the direction of target layer 1
        layer_1_delta = layer_1_error * derive_sigmoid(layer_1)

        synapse_1_weight = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight = (layer_0.T.dot(layer_1_delta))

        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight > 0) + 0) - ((prev_synapse_0_weight > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight > 0) + 0) - ((prev_synapse_1_weight > 0) + 0))
        
        # use alpha as parameter for gradient descent
        synapse_1 += alpha * synapse_1_weight
        synapse_0 += alpha * synapse_0_weight

        prev_synapse_0_weight = synapse_0_weight
        prev_synapse_1_weight = synapse_1_weight

    now = datetime.datetime.now()

    # presist synapse
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
               }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent = 4, sort_keys = True)
    print("saved synapses to: ", synapse_file) 

In [42]:
X = np.array(training)
y = np.array(output)

start_time = time.time()
train(X, y, hidden_neurons = 20, alpha = 0.1, epochs=100000, dropout=False, dropout_rate=0.2)
elapsed_time = time.time() - start_time
print("processing time:", elapsed_time, "seconds")

Training with 20 neurons, alpha:0.1, dropout:False 
Input matrix: 12x26  Output matrix: 1x3
delta after 10000 iterations: 0.00643743377728
delta after 20000 iterations: 0.00439210071396
delta after 30000 iterations: 0.00352153838992
delta after 40000 iterations: 0.00301363524118
delta after 50000 iterations: 0.0026720128549
delta after 60000 iterations: 0.00242254736045
delta after 70000 iterations: 0.00223031278045
delta after 80000 iterations: 0.00207644268251
delta after 90000 iterations: 0.00194974582861
delta after 100000 iterations: 0.00184311358695
saved synapses to:  synapses.json
processing time: 9.651454210281372 seconds


In [43]:
# probability threshold
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json'

with open(synapse_file) as data_file:
    synapse = json.load(data_file)
    synapse_0 = np.asarray(synapse['synapse0'])
    synapse_1 = np.asarray(synapse['synapse1'])

def classify(sentence):
    results = think(sentence)
    
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    results.sort(key = lambda x: x[1], reverse = True)
    return_results = [[classes[r[0]], r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

In [44]:
classify("make me a sandwich")
classify("how are you today?")
classify("talk to you tomorrow")
classify("who are you")
classify("make me some food")
classify("how was your lunch today")
classify("good day")

make me a sandwich 
 classification: [['sandwich', 0.99845244000132216]]
how are you today? 
 classification: [['greeting', 0.99830079959335938]]
talk to you tomorrow 
 classification: [['goodbye', 0.98558125029440802]]
who are you 
 classification: [['greeting', 0.78843085057069684]]
make me some food 
 classification: [['sandwich', 0.6280459351952048]]
how was your lunch today 
 classification: [['greeting', 0.98899662286505952], ['sandwich', 0.25332862095452086]]
good day 
 classification: [['greeting', 0.99661113841568028]]


[['greeting', 0.99661113841568028]]